In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import compress

import datetime
from dateutil.parser import parse

import math
import os
import copy
import pickle

import drugLookup
from sklearn import preprocessing

In [2]:
# Read in data (from pickle file)
file = open('train_set_filtered','rb')
train_set = pickle.load(file)

file = open('test_set_filtered','rb')
test_set = pickle.load(file)

In [3]:
d_train = list(train_set['drug_prediction'])
v_train = list(train_set['vendor_name'])

d_test = list(test_set['drug_prediction'])
v_test = list(test_set['vendor_name'])

In [4]:
# Do time binning: i.e. classify each transaction into given time interval
from datetime import datetime, date, time, timedelta

start = time(0,0,0,0) # first interval start date
interval = 15 # length of interval (minutes)

def addTimes(start_time, duration):
    # Function to add minutes to datetime.time() object
    dt = datetime.combine(date.today(), start_time) # create arbitrary date
    return((dt + timedelta(minutes = duration)).time())

#  Function to check if time is in a time interval
def timeInInterval(start_time, duration, check_time):
    end_time = addTimes(start_time, duration)
    cond1 = start_time <= check_time
    cond2 = check_time < end_time
    return(cond1 and cond2)

# Function to classify given time into one of given bins
def binTime(time1, start_times, duration = interval):
    time_bin = [start_time for start_time in start_times if timeInInterval(start_time, duration, time1.time())]
    if len(time_bin) == 0:
        return(start_times[-1])
    else:
        return(time_bin[0])
    
# Create list of time intervals
# Get list of interval start dates
n_intervals = int(24*60/interval) # compute number of intervals in 24 hour period
slide_amount = interval # amount to slide window (if no overlap desired, set equal to interval)

# Get all intervals in 24 hour period
interval_starts = [addTimes(start_time = start, duration = slide_amount * n) for n in range(n_intervals)]

In [5]:
# For given vendor, count the number of transactions in each time_bin
train_set['time_bin'] = train_set['date'].apply(binTime, start_times = interval_starts)

## Encoding data to numerical values

In [7]:
# Bin time values in the test set
train_set['time_bin'] = train_set['date'].apply(binTime, start_times = interval_starts)

# Get data for testing
X_train = train_set[['drug_prediction','time_bin']]

vendor_le = preprocessing.LabelEncoder()
vendor_le.fit(train_set['vendor_name'])
y_train = vendor_le.transform(train_set['vendor_name'])

# One-hot encoding
enc = preprocessing.OneHotEncoder(categories = 'auto')
enc.fit(X_train)
X_train = enc.transform(X_train)

In [10]:
train_set[['drug_prediction','time_bin']]

,drug_prediction,time_bin
10946,ketamine,19:00:00
10911,ketamine,19:00:00
15416,methamphetamine,22:15:00
21600,heroin,02:00:00
10955,ketamine,19:00:00
10947,ketamine,19:00:00
12920,cocaine,20:30:00
20715,ketamine,01:45:00
20840,dmt,01:45:00
10960,ketamine,19:00:00


## Model training

In [67]:
from sklearn.naive_bayes import MultinomialNB

# Train Naïve-Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Evaluating the model

In [68]:
# Bin time values in the test set
test_set['time_bin'] = test_set['date'].apply(binTime, start_times = interval_starts)

# Get data for testing
X_test = test_set[['drug_prediction','time_bin']]
y_test = vendor_le.transform(test_set['vendor_name'])

# One-hot encoding
X_test = enc.transform(X_test)

In [69]:
# Function to get list of indices with max value
def multi_argmax(v, n):
#     v is vector 
#     n is number of maximum indices to return
    max_indices = []
    temp = copy.deepcopy(v)
    while len(max_indices) < n:
        max_idx = np.argmax(temp)
        max_indices.append(max_idx)
        temp[max_idx] = -1
    return(max_indices)

In [70]:
# Function to get the top class predictions using model.predict_proba
def get_top_n(prob_vector, n, encoder = vendor_le):
    max_indices = multi_argmax(prob_vector, n)
    max_vendors = encoder.inverse_transform(max_indices)
    return(max_vendors)

In [71]:
# Get probability predictions for each transaction in the dataset
prob_ar = nb_model.predict_proba(X_test)
vendor_preds = np.apply_along_axis(get_top_n, 1, prob_ar, n = 15)

In [72]:
# Get accuracy
# Compute 'top n' accuracy
def get_top_n_acc(actuals, preds, n):
    q = zip(actuals, preds)
    if n == 1:
        ar = [samp[0] == samp[1][0] for samp in q]
    else:
        ar = [samp[0] in samp[1][:n] for samp in q]
    acc = np.sum(ar)/len(ar)
    return(round(acc,4))

In [74]:
# Naïve-Bayes, sklearn implementation
print('sklearn Naïve-Bayes,',interval,'minute bins:')
print('Top 1 accuracy :',get_top_n_acc(vendor_le.inverse_transform(y_test), vendor_preds, 1))
print('Top 5 accuracy :',get_top_n_acc(vendor_le.inverse_transform(y_test), vendor_preds, 5))
print('Top 10 accuracy:',get_top_n_acc(vendor_le.inverse_transform(y_test), vendor_preds, 10))

sklearn Naïve-Bayes, 15 minute bins:
Top 1 accuracy : 0.5913
Top 5 accuracy : 0.9575
Top 10 accuracy: 0.973


In [58]:
vendor_preds2 = nb_model.predict(X_test)
df = pd.DataFrame({'actuals': vendor_le.inverse_transform(y_test),\
                   'preds': vendor_le.inverse_transform(vendor_preds2)})